In [35]:
import pandas as pd
import csv
import requests
import cv2
from os import path
import urllib.parse
import numpy as np
from PIL import Image
from io import StringIO
from tensorflow.keras.applications.resnet50 import ResNet50
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, TFRobertaModel, pipeline
import os
import math
import tensorflow as tf
from transformers import BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
data = pd.read_csv("../data/data.csv")


MODEL_NAME = "roberta-base"
MAX_LEN = 200

In [54]:
data

,Unnamed: 0.1,Unnamed: 0,product,item_code,store,brand,url,price,item description,image url
0,0,0,Վարդաջուր «Nairian» միցելային,N:001347,Nairian,Nairian,https://4u.am/hy/product/vardajur-nairian-mice...,10500,Հեշտությամբ մաքրեք երկար օրվա հետքերը դեմքից՝ ...,https://static.4u.am/origin/product/1024/00029...
1,1,1,Օծանելիք «Erato Femme Passionnee»,N:018945,Jamharian Perfume,NaN,https://4u.am/hy/product/ocanelik-erato-femme-...,67000,Էրատո թագուհուն նվիրված աֆրոդիզիակ պարունակող ...,https://static.4u.am/origin/product/1024/76nM0...
2,2,2,"Քսուք «Nairian» աչքի, հակատարիքային, երիտասարդ...",N:000696,Nairian,Nairian,https://4u.am/hy/product/acki-ksuk-nairian-hak...,33300,Օգտակար հատկությունները. Բնության զորությամբ լ...,https://static.4u.am/origin/product/1024/00029...
3,3,3,Օծանելիք «Jamharian Collection Blue»,N:018916,Jamharian Perfume,NaN,https://4u.am/hy/product/ocanelik-jamharian-co...,45000,JC Blue-ն գրավում է ցիտրուսային և ծաղկուն նոտա...,https://static.4u.am/origin/product/1024/7kp16...
4,4,4,Քսուք-բալզամ «Բնատուր» աչքերի շուրջ մաշկի համար,N:000466,Bnatur,Bnatur,https://4u.am/hy/product/ksuk-balzam-bnatur-ac...,7700,Սնուցող և հարթեցնող քսուք-բալզամ: \n\n\nՆշանակ...,https://static.4u.am/origin/product/1024/lxjw-...
...,...,...,...,...,...,...,...,...,...,...
252,856,856,Կոմպոզիցիա ''Բատեա'' վարդերով և թրաշուշաններով,N:090517,Flowers Kiosq,NaN,https://4u.am/hy/product/kompozicia-batea-vard...,45000,Ծաղկեփունջը ներառում է՝\n- վարդ\n- պիոնանման վ...,https://static.4u.am/origin/product/1024/4DDgl...
253,857,857,Ծաղկեփունջ «Կորդովա» վարդագույն փնջային վարդերով,N:079749,Bella,NaN,https://4u.am/hy/product/cagkepunj-kordova-var...,56000,Ծաղիկներ՝ փնջային վարդ:\nԵրկարությունը՝ 70 սմ:...,https://static.4u.am/origin/product/1024/txzLY...
254,858,858,Կոմպոզիցիա «Բանարի» փնջային վարդերով և լիմոնիո...,N:090626,Cataleya,NaN,https://4u.am/hy/product/kompozicia-banari-pnj...,17150,"Ծաղիկները՝ փնջային վարդ, լիմոնիում, գիպսոֆիլիա...",https://static.4u.am/origin/product/1024/ticD_...
255,859,859,"Կոմպոզիցիա «Տորոնտո» փնջային վարդերով, քրիզանթ...",N:077320,Bella,NaN,https://4u.am/hy/product/kompozicia-toronto-pn...,30000,"Ծաղիկները՝ փնջային վարդ, վարդ, քրիզանթեմ\n\n\n...",https://static.4u.am/origin/product/1024/spPWh...


In [56]:
for i in data["image url"]:
    img_data = requests.get(i).content
    url_parts = urllib.parse.urlparse(i)
    path_parts = url_parts[2].rpartition('/')
    address = r"C:\Users\Admin\Desktop\marketing_analytics_group_project\Recommender_system_4u.am\images"
    imgSaveDir = path.join(address, path_parts[2])
    with open(imgSaveDir, "wb") as handler:
        handler.write(img_data)

In [36]:
lst = []
for image in os.listdir('../images/'):
    i = Image.open('../images/' + image)
    i=np.array(i)
    lst.append(i)

In [37]:
lst_= [tf.image.resize(img,size=(224,224)) for img in lst]
model = ResNet50(include_top=True,input_shape=(224, 224, 3))

In [38]:
ar = np.empty((1,1000))
for i in range(len(lst_)):
    m = model(tf.expand_dims(lst_[i],0))
    ar = np.concatenate([ar,m])
with open('arr_images.npy', 'wb') as f:
    np.save(f,ar)

In [34]:
name = data["product"]
description = data["item description"]
description.fillna('',inplace=True)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",)

In [36]:
def roberta_encode(texts, tokenizer): 
    ct = len(texts) 
    input_ids = np.ones((ct, MAX_LEN), dtype='int32') 
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32') 
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification 
 
    for k, text in enumerate(texts): 
        # Tokenize 
        tok_text = tokenizer.tokenize(text) 
         
        # Truncate and convert tokens to numerical IDs 
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)]) 
         
        input_length = len(enc_text) + 2 
        input_length = input_length if input_length < MAX_LEN else MAX_LEN 
         
        # Add tokens [CLS] and [SEP] at the beginning and the end 
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32') 
         
        # Set to 1s in the attention input 
        attention_mask[k,:input_length] = 1 
 
    return { 
        'input_word_ids': input_ids, 
        'input_mask': attention_mask, 
        'input_type_ids': token_type_ids 
    }

def build_model(): 
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids') 
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask') 
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids') 

    # Import RoBERTa model from HuggingFace 
    roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME) 
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids) 

    # Huggingface transformers have multiple outputs, embeddings are the first one, 
    # so let's slice out the first position 
    x = x[0] 

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x) 
    model.compile( 
        optimizer=tf.keras.optimizers.Adam(lr=1e-4), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']) 

    return model

In [62]:
tokenizer = RobertaTokenizer.from_pretrained(model_name)
encoded_product_name = roberta_encode(name, tokenizer)
encoded_product_desc = roberta_encode(description, tokenizer)

In [40]:
model = build_model()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 200)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 200)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 200)]        0           []                               
                                                                                                  
 tf_roberta_model_1 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_word_ids[0][0]',         


C:\Users\Admin\anaconda3\envs\marketing_project\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 odel)                          thPoolingAndCrossAt               'input_mask[0][0]',             
                                tentions(last_hidde               'input_type_ids[0][0]']         
                                n_state=(None, 200,                                               
                                 768),                                                            
                                 pooler_output=(Non                                               
                                e, 768),                                                          
                                 past_key_values=No                                               
                                ne, hidden_states=N                                               
                                one, attentions=Non                                               
                                e, cross_attentions                                               
          

In [63]:
output_name = model.predict(encoded_product_name)
output_name_reduced = np.sum(output_name,axis = 2)

output_desc = model.predict(encoded_product_desc)
output_desc_reduced = np.sum(output_desc,axis = 2)

9/9 [==============================] - 165s 18s/step


In [64]:
with open('arr_text_names.npy', 'wb') as f:
    np.save(f,output_name_reduced)

In [65]:
with open('arr_text_descriptions.npy', 'wb') as f:
    np.save(f,output_desc_reduced)

In [101]:
def calculate_price_diff(row_rec,df):
    price_diff = df['price'].apply(lambda x:(np.exp((x-row_rec['price'])/df['price'].max())))
    return price_diff

In [102]:
pr = pd.DataFrame.from_dict({'price':[10000]},)
prices = calculate_price_diff(pr,data)

In [43]:
def make_store_and_brand_binary(row,df):
    store = row['store']
    brand = row['brand']
    store_match = (df['store']==store).astype(int)
    brand_match = (df['brand']==brand).astype(int)
    total_match = pd.concat([store_match,brand_match],axis = 1)
    return total_match

In [44]:
df = make_store_and_brand_binary(data.iloc[0], data)

In [31]:
def compute_cosine(vec_rec,matrix):
    cos_df = cosine_similarity([vec_rec],matrix)
    cos_df = np.squeeze(cos_df)
    
    return cos_df



In [45]:
with open('./arr_text_names.npy','rb') as f:
    arr = np.load(f)
row = arr[0,:]
names = compute_cosine(row,arr)

In [32]:
with open('./arr_text_descriptions.npy','rb') as f:
    arr = np.load(f)
    
row = arr[0,:]
descriptions = compute_cosine(row,arr)

In [50]:
with open('./arr_images.npy','rb') as f:
    arr = np.load(f)
    
row = arr[0,:]
images = compute_cosine(row,arr)

array([0.00000000e+000, 2.45258642e-312, 1.42107606e-311, 1.45431184e-311,
       2.64531377e-311, 1.39347374e-311, 2.07857279e-311, 3.15870216e-312,
       2.59044553e-313, 1.04230583e-312, 5.49958858e-312, 1.46570798e-311,
       2.62454174e-311, 3.67755009e-313, 1.32880939e-312, 2.92565144e-312,
       6.19578516e-312, 2.96090817e-312, 3.35989809e-313, 1.55577750e-311,
       2.90637570e-311, 1.34748556e-311, 1.02150997e-312, 6.40230524e-313,
       4.17024555e-312, 1.36059481e-311, 3.33144569e-312, 1.79382296e-311,
       1.21012325e-311, 1.70296947e-311, 5.44383408e-312, 8.29373442e-313,
       2.46777478e-311, 2.06360459e-311, 3.31272625e-311, 1.63063411e-312,
       9.52438288e-313, 2.38819907e-311, 4.58404125e-312, 6.63940875e-312,
       4.50966680e-314, 5.08347630e-314, 2.42250892e-311, 1.39764116e-311,
       1.32091346e-312, 1.72914314e-312, 2.88185580e-311, 2.62152132e-313,
       2.44266802e-312, 9.15879900e-312, 1.22451224e-311, 3.34138268e-311,
       2.94242825e-312, 1

In [79]:
data_ = data[~data["image url"].duplicated()]['image url']
image_to_cosine_sim_dict = {image_name:img_array for image_name,img_array in zip(data_.to_numpy(),images)}
df['image similarities'] = data['image url'].map(image_to_cosine_sim_dict)

In [103]:
df["names"] = names
df["descriptions"] = descriptions
df["prices"] = prices

In [83]:
from sklearn.preprocessing import MinMaxScaler

In [88]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df,),columns=df.columns)

In [104]:
df

,store,brand,names,descriptions,image similarities,weighted_sum,prices
0,1.0,1.0,1.000000,1.000000,0.000000e+00,0.800000,1.000878
1,0.0,0.0,0.999373,0.991958,2.452586e-312,0.242990,1.105171
2,1.0,1.0,0.997103,0.992155,1.421076e-311,0.552375,1.041724
3,0.0,0.0,0.999229,0.993519,1.454312e-311,0.262491,1.063328
4,0.0,0.0,0.997554,0.991202,2.645314e-311,0.155464,0.995973
...,...,...,...,...,...,...,...
252,0.0,0.0,0.997221,0.992493,1.446698e-311,0.162770,1.063328
253,0.0,0.0,0.997719,0.991758,1.633772e-311,0.171355,1.084048
254,0.0,0.0,0.996934,0.991686,1.192822e-312,0.137752,1.012623
255,0.0,0.0,0.995984,0.992622,7.603015e-312,0.113737,1.035711


In [105]:
weights = [1/6]*6
df['weighted_sum'] = df["store"]*weights[0] + df["brand"]*weights[1] +df["names"]*weights[2] +df["descriptions"]*weights[3] +df["image similarities"]*weights[4] + df["prices"]*weights[5] 

In [120]:
df["weighted_sum"].sort_values(ascending = False)

172    0.840531
2      0.838497
176    0.836611
29     0.834778
0      0.833480
         ...   
166    0.495971
11     0.495967
134    0.495817
132    0.495506
135    0.495271
Name: weighted_sum, Length: 257, dtype: float64

In [116]:
top5 = df["weighted_sum"].sort_values(ascending = False).index[:5]
data.iloc[top5]

,Unnamed: 0.1,Unnamed: 0,product,item_code,store,brand,url,price,item description,image url,cosine similarities,image similarities
172,501,501,Շիճուկ «Nairian» գիշերային,N:001401,Nairian,Nairian,https://4u.am/hy/product/sicuk-nairian-giserayin,37700,"Սնուցող եւ երիտասարդացնող, հակատարիքային: Բարե...",https://static.4u.am/origin/product/1024/00029...,2.577756e-312,2.577756e-312
2,2,2,"Քսուք «Nairian» աչքի, հակատարիքային, երիտասարդ...",N:000696,Nairian,Nairian,https://4u.am/hy/product/acki-ksuk-nairian-hak...,33300,Օգտակար հատկությունները. Բնության զորությամբ լ...,https://static.4u.am/origin/product/1024/00029...,1.421076e-311,1.421076e-311
176,505,505,"Քսուք «Nairian» աչքի, հանգստացնող և պաշտպանող ...",N:000698,Nairian,Nairian,https://4u.am/hy/product/acki-ksuk-nairian-han...,26600,Զգայուն մաշկի համար: Օգտակար հատկությունները. ...,https://static.4u.am/origin/product/1024/00029...,2.094367e-312,2.094367e-312
29,29,29,Նվեր-տուփ «Nairian» Նարդոսի քնքշություն,N:026320,Nairian,Nairian,https://4u.am/hy/product/nver-tup-nairian-nard...,20100,Նարդոսի քնքշություն հավաքածուն սիրո և հոգատարո...,https://static.4u.am/origin/product/1024/KjoLt...,1.702969e-311,1.702969e-311
0,0,0,Վարդաջուր «Nairian» միցելային,N:001347,Nairian,Nairian,https://4u.am/hy/product/vardajur-nairian-mice...,10500,Հեշտությամբ մաքրեք երկար օրվա հետքերը դեմքից՝ ...,https://static.4u.am/origin/product/1024/00029...,0.000000e+00,0.000000e+00


In [118]:
print(data.iloc[0])

Unnamed: 0.1                                                           0
Unnamed: 0                                                             0
product                                    Վարդաջուր «Nairian» միցելային
item_code                                                       N:001347
store                                                            Nairian
brand                                                            Nairian
url                    https://4u.am/hy/product/vardajur-nairian-mice...
price                                                              10500
item description       Հեշտությամբ մաքրեք երկար օրվա հետքերը դեմքից՝ ...
image url              https://static.4u.am/origin/product/1024/00029...
cosine similarities                                                  0.0
image similarities                                                   0.0
Name: 0, dtype: object
